## Connect to MYSQL and create a table to store our tweets

In [1]:
import tweepy as tw
import mysql.connector
import re
from textblob import TextBlob

# API keys
consumer_key = '33sOuUOYVSkH5aEOlageHz0dA'
consumer_key_secret = 'l14ovpLo3j2Qn5y62OVBOvAQ5MUD2SO8CxLFIwsKORdfB2VcVu'
access_token = '1342134650886565888-YXPBtB7cGc01hUQKZz8RKn7XP56hhF'
access_token_secret = 'wPdvM0HOiu3aAGJGGvYG4nIafoP56nU1IPp4I4WzbzJ07'


# Connection with MySQL
db = mysql.connector.connect(host="localhost", user="root", password="12345678", database="cryptocurrency")
cur = db.cursor(buffered=True) #create a cursor object


# We are gonna create a table named tweets so first we check if table exists
check = "SHOW TABLES LIKE 'tweets'"  
cur.execute(check)


# Create a table if it does not exist in the database
result = cur.fetchone()


if result:
    print('The table tweets already exists')
else:  
    create_table = "CREATE TABLE tweets (ID int(5) NOT NULL AUTO_INCREMENT,username varchar(30) NOT NULL,user_description varchar(50),number_of_followers int(10) NOT NULL,number_of_tweets int(10) NOT NULL,text varchar(1000) NOT NULL,number_of_retweets int(10) NOT NULL,favourites int(10) NOT NULL,date datetime NOT NULL,location varchar(30) NOT NULL,hashtags varchar(300) NOT NULL,sentiment varchar(10) NOT NULL,subjectivity varchar(10) NOT NULL,PRIMARY KEY (ID));"
    cur.execute(create_table)
    db.commit()
    cur.execute("SHOW TABLES")

The table tweets already exists


## Define  appropriate functions to clean the text and to classify the tweets according to the sentiment and to the subjectivity

In [2]:
# Text Preprocessing
def clean_func(tweet_text):
        emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
        tweet_text = emoji_pattern.sub(r'', tweet_text)
        tweet_text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', tweet_text) # Remove punctuation
        tweet_text=re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)","",tweet_text) # Remove Urls
        tweet_text = re.sub(u'\u201c', '', tweet_text)
        return tweet_text
    
    
# Cluster tweets according to their polarity as positive, Neutral,Negative    
def pol(tweet_text):
    if TextBlob(tweet_text).sentiment.polarity > 0:
        polarity = 'Positive'
    elif TextBlob(tweet_text).sentiment.polarity == 0: 
        polarity = 'Neutral'
    else:
        polarity = 'Negative'
    return polarity

          
# Cluster tweets according to their subjectivity as objective, neutral, subjective            
def subj(tweet_text):
    if TextBlob(tweet_text).sentiment.subjectivity > 0 and TextBlob(tweet_text).sentiment.subjectivity < 0.5:
        subjectivity = 'Objective'
    elif TextBlob(tweet_text).sentiment.subjectivity == 0.5: 
        subjectivity = 'Neutral'
    else:
        subjectivity = 'Subjective'
    return subjectivity

In [3]:
# Authenticate Access
auth = tw.OAuthHandler(consumer_key, consumer_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth)

# Fetch all what exist in the table
cur.execute("SELECT * FROM tweets")
database = cur.fetchall()

# Collect tweets that contain the hashtag Bitcoin

In [4]:
## BITCOIN ##
search_Bitcoin = "#Bitcoin" + " -filter:retweets"
tweets_Bitcoin = tw.Cursor(api.search_tweets, tweet_mode="extended", q=search_Bitcoin, lang="en").items(200)

count = 0 # in order to count how many tweets we insert to the database

for tweet in tweets_Bitcoin:

    text = clean_func(tweet.full_text)  # Tweet text
    username = tweet.user.screen_name  # Username
    user_description=clean_func(tweet.user.description) # The description of the user
    number_of_followers = tweet.user.followers_count  # Number of follower's account
    number_of_tweets = tweet.user.statuses_count  # Total number of tweets of this account
    number_of_retweets = tweet.retweet_count  # Total number of account's retweets
    favourites = tweet.favorite_count  # Likes of the tweet
    date = tweet.created_at  # Date that tweet was created.
    location = tweet.user.location  # Location that tweet was created.
    hashtag_list = re.findall(r"#(\w+)", tweet.full_text)  # find all the hashtags in a tweet 
    hashtags = ' '.join([str(elem) for elem in hashtag_list])  # Hashtags of the tweet 
    polarity = pol(text)  # Polarity of the tweet 
    subject=subj(text) # Subjectivity of the tweet

    
    # Insert the data in the database
    insert = "INSERT INTO tweets (username,user_description, number_of_followers, number_of_tweets, text, number_of_retweets, favourites, date, location, hashtags, sentiment,subjectivity) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s)"
    values = (username,user_description,number_of_followers, number_of_tweets, text, number_of_retweets,favourites, date, location, hashtags, polarity,subject)
    
    
    if values not in database:  # avoid dublicates
        try:
            cur.execute(insert, values)
            db.commit()
            count = count + cur.rowcount 
            print(count, "records inserted.")
        except Exception: 
            pass

1 records inserted.
2 records inserted.
3 records inserted.
4 records inserted.
5 records inserted.
6 records inserted.
7 records inserted.
8 records inserted.
9 records inserted.
10 records inserted.
11 records inserted.
12 records inserted.
13 records inserted.
14 records inserted.
15 records inserted.
16 records inserted.
17 records inserted.
18 records inserted.
19 records inserted.
20 records inserted.
21 records inserted.
22 records inserted.
23 records inserted.
24 records inserted.
25 records inserted.
26 records inserted.
27 records inserted.
28 records inserted.
29 records inserted.
30 records inserted.
31 records inserted.
32 records inserted.
33 records inserted.
34 records inserted.
35 records inserted.
36 records inserted.
37 records inserted.
38 records inserted.
39 records inserted.
40 records inserted.
41 records inserted.
42 records inserted.
43 records inserted.
44 records inserted.
45 records inserted.


# Collect tweets that contain the hashtag Ethereum

In [5]:
## ETHEREUM ## 

search_Ethereum='#Ethereum' + " -filter:retweets"
tweets_Ethereum= tw.Cursor(api.search_tweets, tweet_mode="extended", q=search_Ethereum, lang="en").items(500)

count = 0     

for tweet in tweets_Ethereum:

    text = clean_func(tweet.full_text)  # Tweet text
    username = tweet.user.screen_name  # Username
    user_description=clean_func(tweet.user.description) # The description of the user
    number_of_followers = tweet.user.followers_count  # Number of follower's account
    number_of_tweets = tweet.user.statuses_count  # Total number of tweets of this account
    number_of_retweets = tweet.retweet_count  # Total number of account's retweets
    favourites = tweet.favorite_count  # Likes of the tweet
    date = tweet.created_at  # Date that tweet was created.
    location = tweet.user.location  # Location that tweet was created.
    hashtag_list = re.findall(r"#(\w+)", tweet.full_text)  # find all the hashtags in a tweet 
    hashtags = ' '.join([str(elem) for elem in hashtag_list])  # Hashtags of the tweet 
    polarity = pol(text)  # Polarity of the tweet 
    subject=subj(text) # Subjectivity of the tweet

    
  
    # Insert the data in the database
    insert = "INSERT INTO tweets (username,user_description, number_of_followers, number_of_tweets, text, number_of_retweets, favourites, date, location, hashtags, sentiment,subjectivity) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s)"
    values = (username,user_description,number_of_followers, number_of_tweets, text, number_of_retweets,favourites, date, location, hashtags, polarity,subject)
    
    if values not in database:  # avoid dublicates
        try:
            cur.execute(insert, values)
            db.commit()
            count = count + cur.rowcount 
            print(count, "records inserted.")
        except Exception:  
            pass

1 records inserted.
2 records inserted.
3 records inserted.
4 records inserted.
5 records inserted.
6 records inserted.
7 records inserted.
8 records inserted.
9 records inserted.
10 records inserted.
11 records inserted.
12 records inserted.
13 records inserted.
14 records inserted.
15 records inserted.
16 records inserted.
17 records inserted.
18 records inserted.
19 records inserted.
20 records inserted.
21 records inserted.
22 records inserted.
23 records inserted.
24 records inserted.
25 records inserted.
26 records inserted.
27 records inserted.
28 records inserted.
29 records inserted.
30 records inserted.
31 records inserted.
32 records inserted.
33 records inserted.
34 records inserted.
35 records inserted.
36 records inserted.
37 records inserted.
38 records inserted.
39 records inserted.
40 records inserted.
41 records inserted.
42 records inserted.
43 records inserted.
44 records inserted.
45 records inserted.
46 records inserted.
47 records inserted.
48 records inserted.
4

## Collect tweets that contain the hashtag BNB

In [6]:
## BNB ##
search_Bnb='#BNB'+ "-filter:retweets"
tweets_Bnb = tw.Cursor(api.search_tweets, tweet_mode="extended", q=search_Bnb, lang="en").items(500)

count=0

for tweet in tweets_Bnb:

    text = clean_func(tweet.full_text)  # Tweet text
    username = tweet.user.screen_name  # Username
    user_description=clean_func(tweet.user.description) # The description of the user
    number_of_followers = tweet.user.followers_count  # Number of follower's account
    number_of_tweets = tweet.user.statuses_count  # Total number of tweets of this account
    number_of_retweets = tweet.retweet_count  # Total number of account's retweets
    favourites = tweet.favorite_count  # Likes of the tweet
    date = tweet.created_at  # Date that tweet was created.
    location = tweet.user.location  # Location that tweet was created.
    hashtag_list = re.findall(r"#(\w+)", tweet.full_text) # find all the hashtags in a tweet
    hashtags = ' '.join([str(elem) for elem in hashtag_list])  # Hashtags of the tweet 
    polarity = pol(text)  # Polarity of the tweet 
    subject=subj(text) # Subjectivity of the tweet

    
   
    # Insert the data in the database
    insert = "INSERT INTO tweets (username,user_description, number_of_followers, number_of_tweets, text, number_of_retweets, favourites, date, location, hashtags, sentiment,subjectivity) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s)"
    values = (username,user_description,number_of_followers, number_of_tweets, text, number_of_retweets,favourites, date, location, hashtags, polarity,subject)
     
    
    if values not in database:  # avoid dublicates
        try:
            cur.execute(insert, values)
            db.commit()
            count = count + cur.rowcount 
            print(count, "records inserted.")
        except Exception: 
            pass

1 records inserted.
2 records inserted.
3 records inserted.
4 records inserted.
5 records inserted.
6 records inserted.
7 records inserted.
8 records inserted.
9 records inserted.
10 records inserted.
11 records inserted.
12 records inserted.
13 records inserted.
14 records inserted.
15 records inserted.
16 records inserted.
17 records inserted.
18 records inserted.
19 records inserted.
20 records inserted.
21 records inserted.
22 records inserted.
23 records inserted.
24 records inserted.
25 records inserted.
26 records inserted.
27 records inserted.
28 records inserted.
29 records inserted.
30 records inserted.
31 records inserted.
32 records inserted.
33 records inserted.
34 records inserted.
35 records inserted.
36 records inserted.
37 records inserted.
38 records inserted.
39 records inserted.
40 records inserted.
41 records inserted.
42 records inserted.
43 records inserted.
44 records inserted.
45 records inserted.
46 records inserted.
47 records inserted.
48 records inserted.
4

### Comment

In the end of the code we added a piece of code in order to avoid duplicates. However we realised that it didn't solve this problem in total. The reason is that even we add only values that do not exist already in our database, the date is in a datetime format. So, each second the attribute date changes and each record is considered as unique because the seconds change.
Also tweets get likes constantly and we noticed that in our database we have the same tweets but with only a different number of likes, or followers.  Generally, in our database there are a lot of duplicates and that's why there are so many tweets.

In the next piece of code, before we do the sentiment analysis we remove all these duplicates and we keep only these that have a unique tweet text. So our results are based only in the unique tweets and not in the entire database